In [3]:
import pandas as pd
import math
# 读取数据
df = pd.read_csv('C:\\Python\\Vanyosr_Strategy\\Daily_Signal\\Signal.csv')
# 假设数据有 'ts_code'（股票代码）, 'trade_date'（日期）, 'Signal', 'open', 'close', 'low', 'high', 'Down' 列
df

,ts_code,trade_date,open,high,low,close,MACD_Hist,Selected,Down,factor_turn,return_rate_ewm,Signal,stop
0,000807.SZ,20240401,14.16,14.40,13.94,14.12,NaN,0,NaN,-4.440892e-16,NaN,NaN,NaN
1,000807.SZ,20240402,14.01,14.40,13.94,14.24,NaN,0,NaN,-1.289944e-01,0.846267,NaN,NaN
2,000807.SZ,20240403,14.38,15.58,14.26,15.14,NaN,0,NaN,-2.366124e-01,3.619457,NaN,NaN
3,000807.SZ,20240408,15.05,15.66,14.57,14.72,NaN,0,NaN,-1.904888e-01,1.257383,NaN,NaN
4,000807.SZ,20240409,14.54,14.76,14.13,14.27,NaN,0,NaN,-4.333402e-02,-0.001903,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4195,605117.SH,20250623,50.70,52.76,50.33,52.47,-0.229556,0,0.8947,-2.045471e-01,-2.528014,NaN,NaN
4196,605117.SH,20250624,52.58,52.80,52.00,52.63,0.405682,0,0.6352,-1.442050e-01,-2.258254,NaN,NaN
4197,605117.SH,20250625,52.74,53.05,51.95,52.87,0.837741,0,0.4321,-1.087714e-01,-1.999851,NaN,NaN
4198,605117.SH,20250626,52.76,52.76,51.27,51.36,0.941205,0,0.1035,-1.134955e-01,-2.085355,NaN,NaN


In [4]:
# 初始化账户状态
initial_cash = 100000  # 初始资金
available_cash = initial_cash
portfolio = {}  # 股票持仓
trade_log = []  # 记录交易
total_cash_log = []  # 总资金列表

# 获取所有日期按顺序排列
dates = sorted(df['trade_date'].unique())

# 设置每只股票最小买入股数
MIN_SHARES = 100  # 每次买入最少100股

# 假设的滑点和手续费率
slippage = 0.0002  # 0.02%的滑点
buy_commission_rate = 0.0001  # 买入手续费率 0.01%
sell_commission_rate = 0.0002  # 卖出手续费率 0.02%

# 交易逻辑
for date in dates:
    # 筛选出当天有买入信号的股票
    signals_today = df[(df['trade_date'] == date) & (df['Signal'] == 'B')].copy()

    # 跌幅因子买入逻辑
    # if not signals_today.empty:
    #     signals_today['drop'] = None
        
    #     for stock in signals_today['ts_code']:
    #         stock_data = df[df['ts_code'] == stock].copy()
    #         stock_data = stock_data[stock_data['trade_date'] < date].sort_values(by='trade_date', ascending=False).head(4)
    #         if len(stock_data) == 4:
    #             stock_data['drop'] = stock_data['close'].pct_change().min()
    #             signals_today.loc[signals_today['ts_code'] == stock, 'drop'] = stock_data['drop'].min()

    #     signals_today = signals_today.dropna(subset=['drop'])

    #     if not signals_today.empty:
    #         signals_today = signals_today.sort_values(by='drop').head(3)
    #         stock_count = len(signals_today)
    #         position_size = available_cash / stock_count if stock_count > 0 else 0

    # # 均线收敛因子买入逻辑
    if not signals_today.empty:
            signals_today['factor_1'] = None
            for stock in signals_today['ts_code']:
                stock_data = df[df['ts_code'] == stock].copy()
                # 按照因子 factor_turn 从小到大排序，选择最小的三只股票
                signals_today = signals_today.sort_values(by='return_rate_ewm').head(3)

            # 计算选中股票的数量
            stock_count = len(signals_today)
            position_size = available_cash / stock_count if stock_count > 0 else 0
            
            for idx, row in signals_today.iterrows():
                stock_code = row['ts_code']
                buy_price = row['close']
                
                # 计算滑点后的买入价格
                adjusted_buy_price = buy_price * (1 + slippage)
                
                shares_to_buy = math.floor(position_size / adjusted_buy_price / MIN_SHARES) * MIN_SHARES
                total_cost = shares_to_buy * adjusted_buy_price * (1 + buy_commission_rate)
                
                if shares_to_buy >= MIN_SHARES and total_cost <= available_cash:
                    portfolio[stock_code] = {
                        'buy_price': adjusted_buy_price,
                        'quantity': shares_to_buy,
                        'buy_date': date
                    }
                    available_cash -= total_cost
                    
                    trade_log.append({
                        'date': date,
                        'stock': stock_code,
                        'action': 'buy',
                        'price': adjusted_buy_price,
                        'quantity': shares_to_buy,
                        'cash_remaining': available_cash
                    })
    
    # 卖出逻辑
    for stock in list(portfolio.keys()):
        stock_data = df[(df['ts_code'] == stock) & (df['trade_date'] == date)]
        if not stock_data.empty and stock_data.iloc[0]['Signal'] in ['S', 'T']:
            sell_price = stock_data.iloc[0]['close']
            adjusted_sell_price = sell_price * (1 - slippage)
            quantity = portfolio[stock]['quantity']
            total_sell_value = adjusted_sell_price * quantity * (1 - sell_commission_rate)
            available_cash += total_sell_value
            
            trade_log.append({
                'date': date,
                'stock': stock,
                'action': 'sell',
                'price': adjusted_sell_price,
                'quantity': quantity,
                'cash_remaining': available_cash
            })
            
            del portfolio[stock]
    
    total_stock_value = 0
    for stock in portfolio:
        stock_data = df[(df['ts_code'] == stock) & (df['trade_date'] == date)]
        if not stock_data.empty:
            total_stock_value += portfolio[stock]['quantity'] * stock_data.iloc[0]['close']
    
    total_cash = available_cash + total_stock_value
    
    total_cash_log.append({
        'date': date,
        'total_cash': total_cash
    })

# 打印交易记录
trade_log_df = pd.DataFrame(trade_log)
print(trade_log_df)

# 打印总资金记录
total_cash_df = pd.DataFrame(total_cash_log)
print(total_cash_df)

# 保存交易记录和总资金记录到 CSV文件
trade_log_df.to_csv('C:\\Python\\Vanyosr_Strategy\\Daily_Signal\\trade_log.csv', index=False)
total_cash_df.to_csv('C:\\Python\\Vanyosr_Strategy\\Daily_Signal\\total_cash_log.csv', index=False)


         date      stock action      price  quantity  cash_remaining
0    20240510  600938.SH    buy  29.145828      3400      894.275218
1    20240515  600938.SH   sell  28.304338      3400    97109.777469
2    20240516  600426.SH    buy  29.385876      1600    50087.674128
3    20240516  300815.SZ    buy  13.892778      3400     2847.505384
4    20240521  601156.SH    buy  21.574314       100      689.858241
..        ...        ...    ...        ...       ...             ...
162  20250423  601919.SH    buy  14.272854      7500      806.414025
163  20250516  601919.SH   sell  16.046790      7500   121133.268840
164  20250520  603613.SH    buy  24.214842      5000       46.951419
165  20250523  603613.SH   sell  23.375324      5000   116900.196095
166  20250526  601156.SH    buy  12.942588      9000      405.255765

[167 rows x 6 columns]
         date     total_cash
0    20240401  100000.000000
1    20240402  100000.000000
2    20240403  100000.000000
3    20240408  100000.000000
4  